In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
import warnings
warnings.filterwarnings('ignore')

# Initialise App

In [2]:
app = dash.Dash(__name__)

In [3]:
data = pd.read_csv('/Users/emilyvincett/downloads/File (1).csv')
dev = pd.read_csv('/Users/emilyvincett/Downloads/Dev (1).csv')
time = pd.read_csv('/Users/emilyvincett/Downloads/Time (1).csv')
series = pd.read_csv('/Users/emilyvincett/Downloads/Daily (1).csv')

# Fill null values with 0. 
series['Purchases'].fillna(0,inplace=True)

# Create a new column to represent the cost per 1000 impressions
series['C1000IMP'] = (series['Amount Spent (GBP)']/series['Impressions'])*1000

# Use boolean logic to divide the dataset into 2 cost groups.
# i.e. High = above £10 cost per 1000 impression and Low if otherwise
series['Cost'] = np.where(series['C1000IMP']<10,'Low','High')

In [4]:
def GroupByDay(series,func=np.mean):
    grouped = series[['Day','C1000IMP']].groupby('Day')
    daily = grouped.aggregate(func)
    daily['Days'] = daily.index
    daily.sort_values(by='Days')
    daily['Days'] = pd.to_datetime(daily['Days'])
    start = pd.to_datetime(daily['Days'][0])

    one_month = np.timedelta64(1,'M')
    daily['Months'] = (daily['Days'] - start)/one_month
    return daily

def GroupByCostAndDay(series):
    groups = series.groupby('Cost')
    dailies = {}

    for day,cost in groups:
        dailies[day] = GroupByDay(cost)
    return dailies  

In [5]:
app.layout = html.Div([
    html.H1('Facebook Ad Campaign Performance',style={'text-align':'center'}),
    
    dcc.Dropdown(id = 'select_campaign',
                options = [
                    {'label':'Music Industry','value':'Music Industry'},
                    {'label':'Dig Distribution (Global)','value':'Dig Distribution (Global)'},
                    {'label':'Segmented Audience 3 (11 - feb)','value':'Segmented Audience 3 (11 - feb)'},
                    {'label':'Conversions: Jan 11','value':'Conversions: Jan 11'},
                    {'label':'Conversions: 16 - 34/Distribution','value':'Conversions: 16 - 34/Distribution'},
                    {'label':'The One','value':'The One'},
                    {'label':'Mike Baillie (35 - 44)','value':'Mike Baillie (35 - 44)'},
                    {'label':'Conversions - Jan 6 - Distribution (Copy)','value':'Conversions - Jan 6 - Distribution (Copy)'},
                    {'label':'Conversion - Jan 8 - Brand New','value':'Conversion - Jan 8 - Brand New'},
                    {'label':'Conversions 29','value':'Conversions 29'},
                    {'label':'Dig Distribution (Global) - Lifetime Budget','value':'Dig Distribution (Global) - Lifetime Budget'},
                    {'label':'Conversions: Buyers Lookalike','value':'Conversions: Buyers Lookalike'},
                    {'label':'Industry Contacts - First Run','value':'Industry Contacts - First Run'},
                    {'label':'Conversions: Remarketing','value':'Conversions: Remarketing'},
                    {'label':'Segmented Audience 1 - 08 Feb','value':'Segmented Audience 1 - 08 Feb'},
                    {'label':'Conversions: PageView Lookalike','value':'Conversions: PageView Lookalike'},
                    {'label':'Segmented Audience 2: (10 Feb)','value':'Segmented Audience 2: (10 Feb)'},
                    {'label':'Conversions - Prev Link Clicks','value':'Conversions - Prev Link Clicks'},
                    {'label':'Conversions - Jan 6 - View Content Opt - (Copy)','value':'Conversions - Jan 6 - View Content Opt - (Copy)'},
                    {'label':'Conversions: Jan 28','value':'Conversions: Jan 28'},
                    {'label':'Conversions - Jan 6 - PageViews LL (Copy)','value':'Conversions - Jan 6 - PageViews LL (Copy)'},
                    {'label':'Conversions: Jan 28 - Copy','value':'Conversions: Jan 28 - Copy'},
                    {'label':'Conversion - Jan 9 - Audience Expanded','value':'Conversion - Jan 9 - Audience Expanded'},
                    {'label':'Conversions  - 01 - Feb','value':'Conversions  - 01 - Feb'},
                    {'label':'Conversions - Prev Link Clicks - Copy','value':'Conversions - Prev Link Clicks - Copy'},
                    {'label':'1 - Jan - Conversions - Interest Targeting','value':'1 - Jan - Conversions - Interest Targeting'},
                    {'label':'Males','values':'Males'},
                    {'label':'Conversion - Jan 9 - Landing Page Views','value':'Conversion - Jan 9 - Landing Page Views'},
                    {'label':'Conversions: Insta Data','values':'Conversions: Insta Data'},
                    {'label':'Conversions: 16 - 34/Distribution - Copy','value':'Conversions: 16 - 34/Distribution - Copy'},
                    {'label':'Conversions: 16 - 34/Distribution - Landing Views','value':'Conversions: 16 - 34/Distribution - Landing Views'},
                    {'label':'Conversions 30','value':'Conversions 30'}],
                 multi = False,
                 value = 'Music Industry',
                 style = {'width':'40%'}),
    html.Div(id='output_container',children=[]),
    html.Br(),
    
    dcc.Graph(id = 'revenue',figure={},),
    dcc.Graph(id = 'profit',figure={},),
    dcc.Graph(id='bar',figure={},),
    dcc.Graph(id='scatter',figure={},),
    dcc.Graph(id='scatter2',figure={})
   
   
    
    
])

In [6]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
    Output(component_id='revenue', component_property='figure'),
    Output(component_id='profit', component_property='figure'),
    Output(component_id='bar',component_property='figure'),
    Output(component_id='scatter',component_property='figure'),
    Output(component_id='scatter2',component_property='figure')],
    [Input(component_id='select_campaign', component_property='value')])

def update_graph (Chosen_Campaign):
    
    # Original Data
    data.fillna(0,inplace=True)
    data2 = data.copy()
    data2 = data2[(data2['Gender']!='unknown')&(data2['Age']!='unknown')]
    container = 'The campaign chosen by user is {}'.format(Chosen_Campaign)
    selected = data2[data2['Campaign Name']== Chosen_Campaign]
    selected['Reach Ratio'] = selected['Reach']/selected['Reach'].sum()*100
    selected['Impression Ratio'] = selected['Impressions']/selected['Impressions'].sum()*100
    grouped = selected.groupby(['Age','Gender'])[['Purchases','Cost per Purchase']].sum().reset_index()
    revenue = selected.Purchases.sum()*10
    cost =  selected['Amount Spent (GBP)'].sum()
    sales = selected.Purchases.sum()
    reach_ratio = selected.groupby(['Gender','Age'])[['Reach Ratio']].sum().reset_index()
    
    # Impression Device Data 
    dev.fillna(0,inplace=True)
    dev2 = dev.copy()
    dev2 = dev2[dev2['Campaign Name']==Chosen_Campaign]
    _ = dev2.groupby(['Platform','Impression Device'])[['Landing Page Views']].sum().reset_index()
    fdevice = _[_['Platform']=='facebook']['Impression Device']
    idevice = _[_['Platform']=='instagram']['Impression Device']
    fpageviews = _[_['Platform']=='facebook']['Landing Page Views']
    ipageviews = _[_['Platform']=='instagram']['Landing Page Views']
    
    # Time of Day Data
    time2 = time.copy()
    time2 = time2[time2['Campaign Name']==Chosen_Campaign].sort_values(by='Time of Day (Ad Account Time Zone)')
    xt = time2['Time of Day (Ad Account Time Zone)']
    yt = time2['Impressions']
    
    
    # Daily Data
    series2 = series.copy()
    series2['Purchases'].fillna(0,inplace=True)
    series2['C1000IMP'] = (series2['Amount Spent (GBP)']/series2['Impressions'])*1000
    series2['Cost'] = np.where(series2['C1000IMP']<10,'Low','High')
    day = GroupByDay(series2)
    dailies = GroupByCostAndDay(series2)
    high = dailies['High']['C1000IMP']
    low = dailies['Low']['C1000IMP']
    
    # Exponential Moving Average
    emov_avg_h = pd.DataFrame.ewm(high,span=3).mean()
    emov_avg_l = pd.DataFrame.ewm(low,span=3).mean()
    
    
    # First Figure
    fig = go.Figure()
    fig.add_trace(go.Indicator(
        mode = 'number',
        value = sales,
        title = {'text':'Sales'},
        domain = {'x':[0,0.5],'y':[0,0.5]}))
    
    fig.add_trace(go.Indicator(
        mode = 'number',
        value = revenue,
        title = {'text':'Revenue'},
        domain = {'x':[0,0.5],'y':[0.6,1]}))
    
    fig.add_trace(go.Indicator(
        mode = 'number',
        value = revenue - cost,
        title = {'text':'Profit'},
        domain = {'x':[0.5,1],'y':[0.6,1]}))
    
    fig.add_trace(go.Indicator(
        mode = 'number',
        value = selected.Reach.sum(),
        title = {'text':'Reach'},
        domain = {'x':[0.5,1],'y':[0,0.5]}))
    
    
    # Second Figure
    
    fig1 = make_subplots(rows=1,cols=2,specs=[[{'type':'domain'},{'type':'domain'}]])
    fig1.add_trace(go.Pie(labels=reach_ratio['Age'],values=reach_ratio['Reach Ratio'],name='Age'),1,1)
    fig1.add_trace(go.Pie(labels=reach_ratio['Gender'],values=reach_ratio['Reach Ratio'],name='Gender'),1,2)
    fig1.update_traces(hole=.4, hoverinfo="label+percent+name")
    fig1.update_layout(title_text='Campaign Reach By Demographics')
    
    
    # Third Figure  
    fig2 = go.Figure(data=[
        go.Bar(name='Facebook',x=fdevice,y=fpageviews),
        go.Bar(name='Instagram',x=idevice,y=ipageviews)])
      
    fig2.update_layout(barmode='stack',title_text = 'Landing Page Views By Device Type And Platform')
    
    
    # Fourth Figure
    fig3 = go.Figure()
    fig3.add_trace(
        go.Scatter(x=xt,y=yt,mode='lines+markers'))
    fig3.update_layout(title_text='Impressions Delivered By Time Of Day')
    
    
    # Fifth Figure
    fig4 = make_subplots(
        rows = 1, cols = 2,
        subplot_titles=('Under £10','Over £10'))
    fig4.add_trace(go.Scatter(x=emov_avg_l.index,y=emov_avg_l.values,mode='lines+markers'),
                   row=1,col=1)
    fig4.add_trace(go.Scatter(x=emov_avg_h.index[3:],y=emov_avg_h.values[3:],mode='lines+markers'),
                   row=1,col=2)
    fig4.update_layout(title_text='Time Series of Cost Per 1000 Impressions')
    
    
                 
    
        
    return container, fig, fig1,fig2,fig3,fig4

if __name__=='__main__':
    app.run_server(debug=False,use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Mar/2021 09:17:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:17:38] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:17:38] "GET /apple-touch-icon.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:17:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:17:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:17:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:18:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:18:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:18:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:18:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 09:18:59] "POST /_dash-update-component HTTP/1.1" 200 -
